### Cargar dataframe de Canada

In [16]:
!pip install pyspark==2.4.5

#Crear una sesión en Spark, Creamos los objetos spark y sc
#import pyspark
#from pyspark import SparkContext, SparkConf
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]")) # Se va a ejecutar en el cluster de IBM
spark = SparkSession.builder.getOrCreate()

Exception: Java gateway process exited before sending its port number

In [9]:
import types
import pandas as pd
from botocore.client import Config


def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_81aeda152d764e759c8cab4dcaba9ea4 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='3dMpQkiyzYR2o0cb79RJB7xDNIbefMotJIQsHTV0cIha',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_81aeda152d764e759c8cab4dcaba9ea4.get_object(Bucket='proyecto1-donotdelete-pr-wypoxecy0ajcet',Key='Canada.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_0 = pd.read_excel(body)
df_data_0.head()

ModuleNotFoundError: No module named 'ibm_boto3'

### 1. Agregar las siguientes columnas al dataframe: migración 80s, migración 90s, migracion 2000s, migración 2010s y total migración

In [10]:
df_data_0["80s"] = df_data_0[range(1980,1990)].sum(axis=1)
df_data_0["90s"] = df_data_0[range(1990,2000)].sum(axis=1)
df_data_0["2000s"] =df_data_0[range(2000,2010)].sum(axis=1)
df_data_0["2010s"] = df_data_0[range(2010,2014)].sum(axis=1)
df_data_0["Total migración"] = df_data_0[range(1980,2014)].sum(axis=1)
df_data_0.head()

NameError: name 'df_data_0' is not defined

### 2. Crear un nuevo df con la migración total por cada continente (solo 5 continentes) y convertirlo a Schema

In [10]:
df_data_0["Continente"]=df_data_0["Continente"].replace({'Latin America and the Caribbean':"America", 'Northern America':"America"})
df_data_0.Continente.unique()

array(['Asia', 'Europe', 'Africa', 'Oceania', 'America'], dtype=object)

In [13]:
group = df_data_0.groupby("Continente", as_index=False)["Total migración"].sum()
group

,Continente,Total migración
0,Africa,618948
1,America,1006290
2,Asia,3317794
3,Europe,1410947
4,Oceania,55174


In [36]:
df=pd.DataFrame()
df["Continente"]= group.index
df["Migracion"]= group.values
df.head()

,Continente,Migracion
0,Africa,618948
1,America,1006290
2,Asia,3317794
3,Europe,1410947
4,Oceania,55174


### 3. Crear un schema con los paises de oceania y las columnas de migración 80s, migración 90s, migracion 2000s, migración 2010s y total migración

In [38]:
df2 = df_data_0[df_data_0["Continente"]=="Oceania"][["Pais","80s","90s","2000s","Total migración"]]
df2

,Pais,80s,90s,2000s,Total migración
3,American Samoa,3,2,1,6
9,Australia,4564,6574,8804,23829
61,Fiji,5721,9397,4279,20609
91,Kiribati,3,5,5,15
109,Marshall Islands,0,0,2,2
120,Nauru,2,1,14,18
123,New Caledonia,0,0,3,5
124,New Zealand,2719,2344,3626,10386
131,Palau,0,0,1,1
133,Papua New Guinea,17,16,22,64


In [39]:
from pyspark.sql.types import *
mySchema = StructType([ StructField("Pais", StringType(), True)\
                       ,StructField("80s", IntegerType(), True)\
                       ,StructField("90s", IntegerType(), True)\
                       ,StructField("2000s", IntegerType(), True)\
                       ,StructField("Total migración", IntegerType(), True)])

In [41]:
schema = spark.createDataFrame(df2,schema=mySchema) 
schema.show()

+----------------+----+----+-----+---------------+
|            Pais| 80s| 90s|2000s|Total migración|
+----------------+----+----+-----+---------------+
|  American Samoa|   3|   2|    1|              6|
|       Australia|4564|6574| 8804|          23829|
|            Fiji|5721|9397| 4279|          20609|
|        Kiribati|   3|   5|    5|             15|
|Marshall Islands|   0|   0|    2|              2|
|           Nauru|   2|   1|   14|             18|
|   New Caledonia|   0|   0|    3|              5|
|     New Zealand|2719|2344| 3626|          10386|
|           Palau|   0|   0|    1|              1|
|Papua New Guinea|  17|  16|   22|             64|
|           Samoa|  23|  25|   20|             78|
|           Tonga|  41|  60|   26|            142|
|          Tuvalu|   3|   1|    3|              8|
|         Vanuatu|   0|   5|    6|             11|
+----------------+----+----+-----+---------------+



### 4. Mostrar las estadisticas del Schema

In [46]:
schema.describe("80s").show()

+-------+------------------+
|summary|               80s|
+-------+------------------+
|  count|                14|
|   mean| 935.4285714285714|
| stddev|1935.5988027674346|
|    min|                 0|
|    max|              5721|
+-------+------------------+



In [47]:
schema.describe("90s").show()

+-------+------------------+
|summary|               90s|
+-------+------------------+
|  count|                14|
|   mean|1316.4285714285713|
| stddev| 2945.115636081224|
|    min|                 0|
|    max|              9397|
+-------+------------------+



In [48]:
schema.describe("2000s").show()

+-------+------------------+
|summary|             2000s|
+-------+------------------+
|  count|                14|
|   mean| 1200.857142857143|
| stddev|2613.2968231171685|
|    min|                 1|
|    max|              8804|
+-------+------------------+



In [49]:
schema.describe("Total migración").show()

+-------+-----------------+
|summary|  Total migración|
+-------+-----------------+
|  count|               14|
|   mean|           3941.0|
| stddev|8241.599583351988|
|    min|                1|
|    max|            23829|
+-------+-----------------+



### 5. Mostar las columnas del Schema en una lista

In [50]:
schema.columns

['Pais', '80s', '90s', '2000s', 'Total migración']

### 6. Encontrar el promedio, la desviación estandard, el skewnesss y kurtosis de las columnas: migración 80s, migración 90s, migracion 2000s, migración 2010s y total migración

In [14]:
df3  = df_data_0[["Pais","80s","90s","2000s","2010s","Total migración"]]
df3.head()

,Pais,80s,90s,2000s,2010s,Total migración
0,Afghanistan,3693,15845,30501,8600,58639
1,Albania,9,2568,10799,2323,15699
2,Algeria,1271,13153,37833,17182,69439
3,American Samoa,3,2,1,0,6
4,Andorra,2,6,5,2,15


In [15]:
from pyspark.sql.types import *
mySchema2 = StructType([ StructField("Pais", StringType(), True)\
                       ,StructField("80s", IntegerType(), True)\
                       ,StructField("90s", IntegerType(), True)\
                       ,StructField("2000s", IntegerType(), True)\
                       ,StructField("2010s", IntegerType(), True)\
                       ,StructField("Total migración", IntegerType(), True)])

In [16]:
schema2 = spark.createDataFrame(df3,schema=mySchema2) 
schema2.show()

+-------------------+----+-----+-----+-----+---------------+
|               Pais| 80s|  90s|2000s|2010s|Total migración|
+-------------------+----+-----+-----+-----+---------------+
|        Afghanistan|3693|15845|30501| 8600|          58639|
|            Albania|   9| 2568|10799| 2323|          15699|
|            Algeria|1271|13153|37833|17182|          69439|
|     American Samoa|   3|    2|    1|    0|              6|
|            Andorra|   2|    6|    5|    2|             15|
|             Angola|  50|  285| 1563|  215|           2113|
|Antigua and Barbuda| 291|  315|  235|  140|            981|
|          Argentina|3738| 5686| 8890| 1282|          19596|
|            Armenia|   0|  602| 1755|  953|           3310|
|          Australia|4564| 6574| 8804| 3887|          23829|
|            Austria|1968| 1546| 1071|  400|           4985|
|         Azerbaijan|   0|  238| 1846|  565|           2649|
|            Bahamas| 243|  195|  190|  180|            808|
|            Bahrain|  2

In [22]:
from math import sqrt

prerdd = schema2.select("2000s").rdd
rdd = prerdd.map(lambda x : x[0])
#Promedio
promedio  = rdd.sum()/rdd.count()
print("promedio: ",promedio)
#std
std = sqrt(rdd.map(lambda x:pow(x-promedio,2)).sum()/rdd.count())
print("std: ",std)
#skewness
n = float(rdd.count())
skewness = 1/n*(rdd.map(lambda x : pow(x-promedio,3)).sum()/pow(std,3))
print("skewness: ",skewness)
#kurtosis
kurtosis = 1/n*(rdd.map(lambda x : pow(x-promedio,4)).sum()/pow(std,4))
print("kurtosis: ",kurtosis)

promedio:  12105.538461538461
std:  37351.27146117611
skewness:  6.578321694945052
kurtosis:  52.03153229639145


In [23]:
from math import sqrt

for i in ["80s","90s","2000s","2010s","Total migración"]:
    print("****"+i+"*****")
    prerdd = schema2.select(i).rdd
    rdd = prerdd.map(lambda x : x[0])
    #Promedio
    promedio  = rdd.sum()/rdd.count()
    print("promedio: ",promedio)
    #std
    std = sqrt(rdd.map(lambda x:pow(x-promedio,2)).sum()/rdd.count())
    print("std: ",std)
    #skewness
    n = float(rdd.count())
    skewness = 1/n*(rdd.map(lambda x : pow(x-promedio,3)).sum()/pow(std,3))
    print("skewness: ",skewness)
    #kurtosis
    kurtosis = 1/n*(rdd.map(lambda x : pow(x-promedio,4)).sum()/pow(std,4))
    print("kurtosis: ",kurtosis)

****80s*****
promedio:  5422.9384615384615
std:  17056.038556996504
skewness:  6.720388357767422
kurtosis:  60.169261567078756
****90s*****
promedio:  10021.492307692308
std:  29054.457718685164
skewness:  5.718993636936428
kurtosis:  41.14800742628708
****2000s*****
promedio:  12105.538461538461
std:  37351.27146117611
skewness:  6.578321694945052
kurtosis:  52.03153229639145
****2010s*****
promedio:  5317.4820512820515
std:  16844.100211096844
skewness:  6.447762724875128
kurtosis:  47.62528117989149
****Total migración*****
promedio:  32867.45128205128
std:  91549.84874981658
skewness:  5.387819718160344
kurtosis:  34.52967059529177


### 7. Obtener los valores maximos de las columnas

In [28]:
for i in ["80s","90s","2000s","2010s","Total migración"]:
    maxi = schema2.agg({i: "max"}).collect()[0][0]
    print("max "+ i,maxi)


max 80s 179171
max 90s 261966
max 2000s 340385
max 2010s 139241
max Total migración 691904


### 8. Cambiar el nombre de los paises a mayusculas

In [35]:
schema2.select("Pais").rdd.map(lambda x:x[0].upper()).collect()

['AFGHANISTAN',
 'ALBANIA',
 'ALGERIA',
 'AMERICAN SAMOA',
 'ANDORRA',
 'ANGOLA',
 'ANTIGUA AND BARBUDA',
 'ARGENTINA',
 'ARMENIA',
 'AUSTRALIA',
 'AUSTRIA',
 'AZERBAIJAN',
 'BAHAMAS',
 'BAHRAIN',
 'BANGLADESH',
 'BARBADOS',
 'BELARUS',
 'BELGIUM',
 'BELIZE',
 'BENIN',
 'BHUTAN',
 'BOLIVIA (PLURINATIONAL STATE OF)',
 'BOSNIA AND HERZEGOVINA',
 'BOTSWANA',
 'BRAZIL',
 'BRUNEI DARUSSALAM',
 'BULGARIA',
 'BURKINA FASO',
 'BURUNDI',
 'CABO VERDE',
 'CAMBODIA',
 'CAMEROON',
 'CANADA',
 'CENTRAL AFRICAN REPUBLIC',
 'CHAD',
 'CHILE',
 'CHINA',
 'CHINA, HONG KONG SPECIAL ADMINISTRATIVE REGION',
 'CHINA, MACAO SPECIAL ADMINISTRATIVE REGION',
 'COLOMBIA',
 'COMOROS',
 'CONGO',
 'COSTA RICA',
 "CÔTE D'IVOIRE",
 'CROATIA',
 'CUBA',
 'CYPRUS',
 'CZECH REPUBLIC',
 "DEMOCRATIC PEOPLE'S REPUBLIC OF KOREA",
 'DEMOCRATIC REPUBLIC OF THE CONGO',
 'DENMARK',
 'DJIBOUTI',
 'DOMINICA',
 'DOMINICAN REPUBLIC',
 'ECUADOR',
 'EGYPT',
 'EL SALVADOR',
 'EQUATORIAL GUINEA',
 'ERITREA',
 'ESTONIA',
 'ETHIOPIA',
 'F

In [36]:
schema2.show()

+-------------------+----+-----+-----+-----+---------------+
|               Pais| 80s|  90s|2000s|2010s|Total migración|
+-------------------+----+-----+-----+-----+---------------+
|        Afghanistan|3693|15845|30501| 8600|          58639|
|            Albania|   9| 2568|10799| 2323|          15699|
|            Algeria|1271|13153|37833|17182|          69439|
|     American Samoa|   3|    2|    1|    0|              6|
|            Andorra|   2|    6|    5|    2|             15|
|             Angola|  50|  285| 1563|  215|           2113|
|Antigua and Barbuda| 291|  315|  235|  140|            981|
|          Argentina|3738| 5686| 8890| 1282|          19596|
|            Armenia|   0|  602| 1755|  953|           3310|
|          Australia|4564| 6574| 8804| 3887|          23829|
|            Austria|1968| 1546| 1071|  400|           4985|
|         Azerbaijan|   0|  238| 1846|  565|           2649|
|            Bahamas| 243|  195|  190|  180|            808|
|            Bahrain|  2

### 9. Encontrar el pais con más migración en los 90s

In [49]:
schema2.orderBy("90s", ascending=False).select("Pais","90s").take(1)[0][0]

'United Kingdom of Great Britain and Northern Ireland'

In [50]:
schema2.orderBy("90s", ascending=False).select("Pais","90s").take(1)[0][1]

261966

In [52]:
maximo=schema2.agg({"90s": "max"}).collect()
schema2.filter(schema2["90s"]==maximo[0][0]).show()

+--------------------+------+------+-----+-----+---------------+
|                Pais|   80s|   90s|2000s|2010s|Total migración|
+--------------------+------+------+-----+-----+---------------+
|United Kingdom of...|179171|261966|83413|26950|         551500|
+--------------------+------+------+-----+-----+---------------+

